## importação de bibliotecas


In [58]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import pandas as pd
import psycopg2 as pg
import panel as pn


## integração ao banco de dados


In [ ]:

DB_USER = 'postgres'
DB_PASSWORD = 'senha_do_banco'
DB_HOST = 'localhost' 
DB_NAME = 'nome_do_banco'


engine = create_engine(
    f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}',
    connect_args={'options': '-c client_encoding=UTF8'}
)



try:
    with engine.connect() as conn:
        print(" Conectado ao banco de dados!")
except Exception as e:
    print(f" Erro ao conectar: {e}")



 Conectado ao banco de dados!


## testas

In [60]:
import pandas as pd


df_produtos = pd.read_sql("SELECT * FROM produtos", con=engine)


print(df_produtos.head())


   id             nome                      descricao  preco categoria  \
0   1   Camiseta Preta  Camiseta de algodão tamanho M   49.9    Roupas   
1   2  Tênis Esportivo  Tênis para corrida tamanho 42  199.9  Calçados   

                        imagem_url  usuario_id               data_criacao  
0  https://imagem.com/camiseta.jpg           1 2025-02-25 20:17:16.351763  
1     https://imagem.com/tenis.jpg           2 2025-02-25 20:17:16.351763  


## engine


In [61]:
engine = create_engine(
    f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}',
    connect_args={'options': '-c client_encoding=UTF8'}
)


## buscar produto

In [ ]:
def buscar_produtos(query):
    if not query:
        return pn.pane.HTML("<p style='color:gray;'>Digite um nome de produto ou vendedor para pesquisar.</p>")

    query_sql = """
        SELECT p.id, p.nome, p.preco, p.descricao, p.imagem_url, u.nome AS vendedor,
               e.rua, e.cidade, e.estado, e.cep, e.pais
        FROM produtos p
        JOIN usuarios u ON p.usuario_id = u.id
        LEFT JOIN enderecos e ON u.id = e.usuario_id
        WHERE p.nome ILIKE %(query)s OR u.nome ILIKE %(query)s
    """
    
    try:
        df = pd.read_sql(query_sql, con=engine, params={"query": f"%{query}%"})
        df = df.astype(str).applymap(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))

        if df.empty:
            return pn.pane.HTML("<p style='color:red;'>Nenhum resultado encontrado.</p>")
        
        
        cards = []
        for _, row in df.iterrows():
            preco_formatado = f"R$ {float(row['preco']):.2f}" if pd.notna(row["preco"]) else "Preço não disponível"
            
            card = pn.Column(
                f"**{row['nome']}** - {preco_formatado}",
                pn.widgets.Button(name="Expandir", button_type="primary", width=100),
                visible=True,
            )

            botao_adicionar = pn.widgets.Button(name="Adicionar ao Carrinho", button_type="success")

            botao_adicionar.on_click(lambda event, produto_id=row["id"]: adicionar_ao_carrinho(produto_id))

            # Criar string formatada do endereço
            endereco_vendedor = (
                f" {row['rua']}, {row['cidade']}, {row['estado']} - {row['cep']}, {row['pais']}"
                if pd.notna(row["rua"]) else " Endereço não disponível"
            )

            
            
            imagem_url = row['imagem_url'] if row['imagem_url'] and row['imagem_url'].startswith("http") else "https://via.placeholder.com/150"

            detalhes = pn.Column(
                f"**Descrição:** {row['descricao']}",
                pn.pane.Image(imagem_url, width=150),  # 🔹 Agora sempre haverá uma imagem válida
                f"**Vendedor:** {row['vendedor']}",
                f"**Endereço:** {endereco_vendedor}",
                botao_adicionar,
                visible=False,
            )



            def toggle_detalhes(event, detalhes=detalhes):
                detalhes.visible = not detalhes.visible

            card[1].on_click(toggle_detalhes)
            card.append(detalhes)
            cards.append(card)

        return pn.Column(*cards)

    except Exception as e:
        return pn.pane.HTML(f"<p style='color:red;'>Erro ao buscar produtos: {str(e)}</p>")


## add carrinho

In [63]:
search_input = pn.widgets.TextInput(name="Buscar Produto ou Vendedor", placeholder="Digite um nome...")


from sqlalchemy.sql import text 

carrinho_refresh = pn.widgets.Button(name='Atualizar Carrinho', button_type='primary', visible=False)

mensagem_saida = pn.pane.Markdown("")

def adicionar_ao_carrinho(produto_id):
    try:
        with engine.connect() as conn:
            query = text("""
                INSERT INTO carrinho (usuario_id, produto_id, quantidade)
                VALUES (:usuario_id, :produto_id, 1)
                ON CONFLICT (usuario_id, produto_id) 
                DO UPDATE SET quantidade = carrinho.quantidade + 1
            """)
            conn.execute(query, {"usuario_id": 1, "produto_id": produto_id})
            conn.commit()

        carrinho_refresh.clicks += 1
    except Exception:
        pass  #

## exibir carrinho

In [ ]:
def exibir_carrinho(event=None):
    query_sql = """
        SELECT c.id, p.nome AS produto, p.preco, c.quantidade, (p.preco * c.quantidade) AS total
        FROM carrinho c
        JOIN produtos p ON c.produto_id = p.id
    """
    
    try:
        df = pd.read_sql(query_sql, con=engine)

        if df.empty:
            return pn.pane.HTML("<p style='color:gray;'>Carrinho vazio.</p>")

        # 🔹 Garantir que os valores são numéricos
        df["preco"] = pd.to_numeric(df["preco"], errors="coerce")
        df["quantidade"] = pd.to_numeric(df["quantidade"], errors="coerce")
        df["total"] = df["preco"] * df["quantidade"]

        # 🔹 Calcular o total da compra
        total_compra = df["total"].sum()

        # 🔹 Criar botões "Remover" para cada item
        carrinho_items = []
        for _, row in df.iterrows():
            botao_remover = pn.widgets.Button(name="Remover", button_type="danger", width=100)

            # 🔹 Captura corretamente o ID do item ao clicar
            botao_remover.on_click(lambda event, item_id=row["id"]: remover_do_carrinho(item_id))

            item_row = pn.Row(
                f"**{row['produto']}** - R$ {row['preco']:.2f} (Qtd: {row['quantidade']})",
                botao_remover
            )
            carrinho_items.append(item_row)

        # 🔹 Exibir o total da compra no carrinho
        total_label = pn.pane.Markdown(f"**Total da Compra:** R$ {total_compra:.2f}")

        return pn.Column(*carrinho_items, total_label)
    
    except Exception as e:
        return pn.pane.HTML(f"<p style='color:red;'>Erro ao carregar carrinho: {str(e)}</p>")


## remover do carrinho

In [65]:
def remover_do_carrinho(item_id):
    try:
        with engine.connect() as conn:
            query = text("DELETE FROM carrinho WHERE id = :item_id")
            conn.execute(query, {"item_id": item_id})
            conn.commit()

        carrinho_refresh.clicks += 1 
    except Exception:
        pass

## finalizar compra

In [ ]:
def finalizar_compra():
    try:
        
        usuario_id = 1  

        
        endereco_query = text("SELECT id FROM enderecos WHERE usuario_id = :usuario_id LIMIT 1")
        with engine.connect() as conn:
            endereco_result = conn.execute(endereco_query, {"usuario_id": usuario_id}).fetchone()
            endereco_id = endereco_result[0] if endereco_result else None

        
        total_query = text("""
            SELECT SUM(p.preco * c.quantidade) 
            FROM carrinho c 
            JOIN produtos p ON c.produto_id = p.id 
            WHERE c.usuario_id = :usuario_id
        """)
        with engine.connect() as conn:
            total_result = conn.execute(total_query, {"usuario_id": usuario_id}).fetchone()
            total = total_result[0] if total_result[0] else 0

        
        if total == 0:
            return pn.pane.Markdown(" Carrinho vazio! Adicione itens antes de finalizar a compra.")

        
        pedido_query = text("""
            INSERT INTO pedidos (usuario_id, endereco_id, total) 
            VALUES (:usuario_id, :endereco_id, :total)
            RETURNING id
        """)
        with engine.connect() as conn:
            pedido_result = conn.execute(pedido_query, {"usuario_id": usuario_id, "endereco_id": endereco_id, "total": total})
            pedido_id = pedido_result.fetchone()[0]
            conn.commit()

        
        with engine.connect() as conn:
            conn.execute(text("DELETE FROM carrinho WHERE usuario_id = :usuario_id"), {"usuario_id": usuario_id})
            conn.commit()

        
        carrinho_refresh.clicks += 1

        return pn.pane.Markdown(f" Compra finalizada com sucesso! Pedido ID: {pedido_id}")

    except Exception as e:
        return pn.pane.HTML(f"<p style='color:red;'>Erro ao finalizar compra: {str(e)}</p>")


## botão finalizar compra

In [ ]:
botao_finalizar = pn.widgets.Button(name="Finalizar Compra", button_type="success", width=200)
botao_finalizar.on_click(lambda event: finalizar_compra())


dashboard = pn.Row(
    pn.Column(
        "## Conexão Bazar",
        search_input,
        pn.bind(buscar_produtos, search_input),
    ),
    pn.Column(
        "## Carrinho",
        pn.bind(exibir_carrinho, carrinho_refresh),
        botao_finalizar,
        width=300,
    )
)


## exibir pedidos

In [ ]:
def exibir_pedidos():
    query_sql = """
        SELECT p.id, u.nome AS cliente, e.rua, e.cidade, e.estado, p.data_pedido, p.status, p.total
        FROM pedidos p
        JOIN usuarios u ON p.usuario_id = u.id
        LEFT JOIN enderecos e ON p.endereco_id = e.id
        ORDER BY p.data_pedido DESC
    """
    
    try:
        df = pd.read_sql(query_sql, con=engine)

        if df.empty:
            return pn.pane.HTML("<p style='color:gray;'>Nenhum pedido encontrado.</p>")

        df["data_pedido"] = pd.to_datetime(df["data_pedido"]).dt.strftime("%d/%m/%Y %H:%M")

        pedidos_list = []
        for _, row in df.iterrows():
            endereco = f"{row['rua']}, {row['cidade']}, {row['estado']}" if pd.notna(row["rua"]) else "Endereço não informado"
            pedido_card = pn.Column(
                f"**Pedido ID:** {row['id']}",
                f"**Cliente:** {row['cliente']}",
                f"**Endereço:** {endereco}",
                f"**Data:** {row['data_pedido']}",
                f"**Status:** {row['status']}",
                f"**Total:** R$ {row['total']:.2f}",
                pn.layout.Divider(),
            )
            pedidos_list.append(pedido_card)

        return pn.Column(*pedidos_list)

    except Exception as e:
        return pn.pane.HTML(f"<p style='color:red;'>Erro ao carregar pedidos: {str(e)}</p>")


## layout dashboard

In [ ]:

engine = create_engine(
    f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}',
    connect_args={'options': '-c client_encoding=UTF8'}
)
carrinho_refresh = pn.widgets.Button(name='Atualizar Carrinho', button_type='primary', visible=False)

search_input = pn.widgets.TextInput(name="Buscar Produto ou Vendedor", placeholder="Digite um nome...")


botao_finalizar = pn.widgets.Button(name="Finalizar Compra", button_type="success", width=200)


botao_ver_pedidos = pn.widgets.Button(name="Ver Pedidos", button_type="primary", width=200)


painel_pedidos = pn.Column()


def atualizar_pedidos(event):
    painel_pedidos.objects = [exibir_pedidos()]

botao_ver_pedidos.on_click(atualizar_pedidos)


botao_finalizar.on_click(lambda event: finalizar_compra())

dashboard = pn.Row(
    pn.Column(
        "## Conexão Bazar",
        search_input,
        pn.bind(buscar_produtos, search_input),
    ),
    pn.Column(
        "## Carrinho",
        pn.bind(exibir_carrinho, carrinho_refresh),
        botao_finalizar,
        botao_ver_pedidos,
        painel_pedidos,
        width=400,
    )
)



dashboard.servable()

C:\Users\games\AppData\Local\Temp\ipykernel_4696\3711127978.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.astype(str).applymap(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))


Row
    [0] Column
        [0] Markdown(str)
        [1] TextInput(name='Buscar Produto o..., placeholder='Digite um nome...')
        [2] ParamFunction(function, _pane=HTML, defer_load=False)
    [1] Column(width=300)
        [0] Markdown(str)
        [1] ParamFunction(function, _pane=HTML, defer_load=False)